In [ ]:
from math import factorial, exp
import matplotlib.pyplot as plt
import numpy as np
import csv
import time

def offeredTraffic(calls_per_hour, hours_per_call): # A0
    return calls_per_hour*hours_per_call

def erlangB(n, A0):
    denom = 0
    for i in range(n+1):
        denom += (A0**i)/(factorial(i))
    E1 = ((A0**n)/factorial(n))/denom
    return E1

# def erlangC(n, A0):
#     denom = ((A0**n)/factorial(n))*(n/(n-A0))
#     for i in range(n):
#         denom += (A0**i)/factorial(i)
#     return ((A0**n)/factorial(n))*(n/(n-A0))/denom

def erlangC(n, A0):
    denomSum = 0
    for i in range(n):
        denomSum += (A0**i)/factorial(i)
    denom = A0**n + factorial(n)*(1-A0/n)*denomSum
    return (A0**n)/denom

def meanCallDelay(n, A0, meanCallDuration, probDelay):
    # print(probDelay)
    return probDelay*meanCallDuration/(n-A0)

def meanQueuedDelay(n, A0, meanCallDuration):
    return meanCallDuration/(n-A0)

def erlangC_allowedDelayGOS(n, A0, meanCallDuration, allowedDelay=0):
    probDelay = erlangC(n,A0)
    return probDelay*exp(-(n-A0)*allowedDelay/meanCallDuration)


In [ ]:
def trafficSimulation(numChannels, numCalls, meanCallDuration, requeue=False, recheckChannelsDelay=0, allowedDelay=0, holdingDistrib='exp', repetitions=1):

    callFailRates = np.zeros(repetitions)
    meanCallDelay = np.zeros(repetitions)

    for i in range(repetitions):

        if holdingDistrib == 'exp':
            callDurations = np.random.exponential(scale=meanCallDuration, size=numCalls)

        elif holdingDistrib == 'gamma':
            # callDurations = np.random.gamma(shape=1, scale=meanCallDuration, size=numCalls)
            callDurations = np.random.gamma(shape=1.2073, scale=0.03205, size=numCalls)  # shape from paper, scale from paper: 0.03205 = muCHT*(muPaper/bPaper) = muCHT * (29.6121/35.875)

        callStarts = np.random.uniform(size=numCalls)
        # callStarts = np.random.poisson(lam=1/(meanCallDuration*3600), size=numCalls)
        # plt.hist(callStarts, 50)
        # plt.show()
        callStarts.sort()

        # Calls = Numpy array of [[call0_start call0_end]; [call1_start call1_end];...
        calls = np.stack((callStarts, np.add(callStarts,callDurations)),axis=1)

        channelFreeTimes = np.zeros(numChannels)
        time = 0
        callsFailed = 0
        totalDelay = 0
        for call in calls:
            time = call[0]
            channelFound = False

            for j in range(numChannels):
                if not(channelFound):
                    if channelFreeTimes[j] <= time:
                        channelFreeTimes[j] = call[1]
                        channelFound = True

            if channelFound == False:
                if requeue == False:
                    callsFailed += 1

        ## add max wait time?_______________________
                elif recheckChannelsDelay > 0:
                    while channelFound == False:
                        time += recheckChannelsDelay
                        totalDelay += recheckChannelsDelay

                        call[1] += recheckChannelsDelay # move forward call end time
                        for j in range(numChannels):
                            if not(channelFound): # Stop checking channels as soon as a free one is found
                                if channelFreeTimes[j] <= time:
                                    channelFreeTimes[j] = call[1]
                                    channelFound = True
                        
                else:
                    nextCallFinishTime = channelFreeTimes.min()
                    totalDelay += nextCallFinishTime-call[0]
                    call[1] += nextCallFinishTime-call[0]
                    if nextCallFinishTime-call[0]>allowedDelay: # only count call as failed if above the allowed wait time (if allowance present)
                        callsFailed += 1
                    indices = np.where(channelFreeTimes == nextCallFinishTime)
                    index = indices[0][0]
                    channelFreeTimes[index] = call[1]

        callFailRates[i] = callsFailed/numCalls
        meanCallDelay[i] = totalDelay/numCalls

    if requeue == False:
        return callFailRates.mean(), np.std(callFailRates, ddof=1)
    else:
        return callFailRates.mean(), np.std(callFailRates, ddof=1), meanCallDelay.mean(), np.std(meanCallDelay, ddof=1)

In [ ]:

meanCallDuration = 2.33/60# 2.33 mins in hrs
n = 57

# simulationCallAmounts = np.linspace(100, 3000, 30).astype(np.int)
# simulationCallAmounts = np.linspace(0, 2000, 81).astype(np.int)
simulationCallAmounts = (np.linspace(1, 100, 50)/meanCallDuration).astype(np.int)
A0s = offeredTraffic(simulationCallAmounts, meanCallDuration)
repetitionsPerSimulation = 1000
print(simulationCallAmounts, A0s)

In [ ]:
# CALLS BLOCKED - NO REQUEUING
startTime= time.time()
meanCallDuration = 2.33/60# 2.33 mins in hrs
n = 57

simulationCallAmounts = (np.linspace(1, 100, 50)/meanCallDuration).astype(np.int)
A0s = offeredTraffic(simulationCallAmounts, meanCallDuration)
repetitionsPerSimulation = 1000

# erlangBs = np.zeros(simulationCallAmounts.shape)
# GOS_exp = np.zeros(simulationCallAmounts.shape)
# stdDevs_exp = np.zeros(simulationCallAmounts.shape)
# GOS_gamma = np.zeros(simulationCallAmounts.shape)
# stdDevs_gamma = np.zeros(simulationCallAmounts.shape)

# print("# Calls:\tTraffic:\tErlang B:\t\tSim GOS (exp):\t\tstdDev (exp):\t\tSim GOS (gamma)\t\tstdDev (gamma):")
# for i, numCalls in enumerate(simulationCallAmounts):

    # A0 = A0s[i]#offeredTraffic(numCalls, meanCallDuration)
    # erlangBs[i] = erlangB(n, A0)*100

#     meanBlockingRate_exp, stdDev_exp = trafficSimulation(n, numCalls, meanCallDuration, holdingDistrib='exp', repetitions=repetitionsPerSimulation)
#     GOS_exp[i] = meanBlockingRate_exp*100
#     stdDevs_exp[i] = stdDev_exp*100

# #     meanBlockingRate_gamma, stdDev_gamma = trafficSimulation(n, numCalls, meanCallDuration, holdingDistrib='gamma', repetitions=repetitionsPerSimulation)
# #     GOS_gamma[i] = meanBlockingRate_gamma*100
# #     stdDevs_gamma[i] = stdDev_gamma*100

#     print("{}\t\t{:.4f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}".format(numCalls, A0, erlangBs[i], GOS_exp[i], stdDevs_exp[i], GOS_gamma[i], stdDevs_gamma[i]))

plt.figure(figsize=(8,6))
markerSize=2
lineWidth=1
plt.plot(A0s, erlangBs, label='Erlang B', color="black", marker='o', markersize=markerSize, linewidth=lineWidth)

plt.plot(A0s, GOS_exp, label="Monte Carlo (Exponential)", color="red", marker='o', markersize=markerSize, linewidth=lineWidth)
# plt.errorbar(trafficRange, GOS_exp, yerr=stdDevs_exp, label="Monte Carlo (Exponential)", color="red")
plt.fill_between(A0s, GOS_exp+stdDevs_exp, GOS_exp-stdDevs_exp, interpolate=True, color="red", alpha=0.15, label="Monte Carlo +/-1 Standard Deviation")

# plt.errorbar(trafficRange, GOS_gamma, yerr=stdDevs_gamma, label="Gamma", color="green")
# plt.fill_between(trafficRange, GOS_gamma+stdDevs_gamma, GOS_gamma-stdDevs_gamma, interpolate=True, color="green", alpha=0.2)

plt.xlabel('Offered traffic (Erlangs)')
plt.ylabel('Grade Of Service (% calls blocked)')
plt.title("GOS vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.grid()
plt.savefig("sim1.png")
plt.show()

timeTaken = time.time()-startTime
print(timeTaken)

In [ ]:
# ERROR
error = erlangBs-GOS_exp
print(erlangBs[40])
print(GOS_exp[40])
print(erlangBs[40]-GOS_exp[40])
print(error[40])

plt.figure(figsize=(8,6))

plt.plot(A0s, error, label="Error", color='red')
plt.vlines(x = A0s[n//2], ymin = 0, ymax = error[n//2], color = 'blue', label = 'Offered Traffic = 57 = # channels', linewidth=1)

plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('GOS Error (% probability of a call being blocked)')
plt.title("GOS Error (Erlang B - Monte Carlo) vs Offered Traffic ({} channels)".format(n))
plt.ylim([0,2])
plt.legend()
plt.grid()
plt.savefig('sim1_2.png')
plt.show()

print(error.mean(), absError.mean(), absError.mean()-error.mean())

In [ ]:
meanCallDuration = 2.33/60# 2.33 mins in hrs
n = 57

simulationCallAmounts = (np.linspace(1, 58, 58)/meanCallDuration).astype(np.int)
A0s = offeredTraffic(simulationCallAmounts, meanCallDuration)
repetitionsPerSimulation = 500
# print(simulationCallAmounts, A0s)

In [ ]:
# WITH REQUEUING

erlangBs = np.zeros(simulationCallAmounts.shape)
erlangCs = np.zeros(simulationCallAmounts.shape)
meanCallDelays_calc = np.zeros(simulationCallAmounts.shape)

qGOS_exp = np.zeros(simulationCallAmounts.shape)
qStdDevs_exp = np.zeros(simulationCallAmounts.shape)
meanCallDelays_exp = np.zeros(simulationCallAmounts.shape)
callDelayStdDevs_exp = np.zeros(simulationCallAmounts.shape)

# GOS_gamma = np.zeros(simulationCallAmounts.shape)
# stdDevs_gamma = np.zeros(simulationCallAmounts.shape)
# meanCallDelays_gamma = np.zeros(simulationCallAmounts.shape)
# callDelayStdDevs_gamma = np.zeros(simulationCallAmounts.shape)

allowedDelay = 0 # (in hours)
recheckDelay = 0 #20/3600 # (in hours)

print("# Calls:\tTraffic:\tErlang C:\t\tSim GOS (exp):\t\tSim GOS stdDev\t\tCalc. delay/call\tSim Mean delay/call\tdelay/call stdDev")
for i, numCalls in enumerate(simulationCallAmounts):

    A0 = A0s[i]
    erlangBs[i] = erlangB(n, A0)*100
    erlangCs[i] = erlangC_allowedDelayGOS(n, A0, meanCallDuration, allowedDelay=allowedDelay)*100
    # meanCallDelays_calc[i] = meanCallDelay(n, A0, meanCallDuration, np.clip(erlangCs/100,0,1)[i])

    meanDelayRate_exp, stdDev_exp,  meanCallDelay_exp, callDelayStdDev_exp = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, recheckChannelsDelay=recheckDelay, allowedDelay=allowedDelay, holdingDistrib='exp', repetitions=repetitionsPerSimulation)
    qGOS_exp[i] = meanDelayRate_exp*100
    qStdDevs_exp[i] = stdDev_exp*100
    # meanCallDelays_exp[i] = meanCallDelay_exp
    # callDelayStdDevs_exp[i] = callDelayStdDev_exp

    # meanDelayRate_gamma, stdDev_gamma,  meanCallDelay_gamma, callDelayStdDev_gamma = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, recheckChannelsDelay=recheckDelay, allowedDelay=allowedDelay, holdingDistrib='gamma', repetitions=repetitionsPerSimulation)
    # GOS_gamma[i] = meanDelayRate_gamma*100
    # stdDevs_gamma[i] = stdDev_gamma*100
    # meanCallDelays_gamma[i] = meanCallDelay_gamma
    # callDelayStdDevs_gamma[i] = callDelayStdDev_gamma

    print("{}\t\t{:.4f}\t\t\t{:.3f}\t\t\t{:.3f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}".format(numCalls, A0, erlangCs[i], qGOS_exp[i], qStdDevs_exp[i], meanCallDelays_calc[i], meanCallDelays_exp[i], callDelayStdDevs_exp[i]))

trafficRange = offeredTraffic(simulationCallAmounts,meanCallDuration)
plt.figure(figsize=(12,6))

markerSize=4
plt.plot(trafficRange, np.clip(erlangCs,0,100), label='Erlang C', color="orange", marker='o', markersize=markerSize)
plt.plot(trafficRange, erlangBs, label='Erlang B', color="black", marker='o', markersize=markerSize)

plt.plot(trafficRange, qGOS_exp, label="Monte Carlo with requeue (Exponential)", color="red", marker='x', markersize=markerSize)
# plt.errorbar(trafficRange, qGOS_exp, yerr=qStdDevs_exp, label="Monte Carlo with requeue (Exponential)", color="red")
plt.fill_between(trafficRange, qGOS_exp+qStdDevs_exp,  qGOS_exp-qStdDevs_exp, interpolate=True, color="red", alpha=0.15, label="Monte Carlo with requeue +/-1 Standard Deviation")

# plt.errorbar(trafficRange, GOS_gamma, yerr=stdDevs_gamma, label="Monte Carlo with requeue (Gamma)", color="green")
# plt.fill_between(trafficRange, GOS_gamma+stdDevs_gamma, GOS_gamma-stdDevs_gamma, interpolate=True, color="green", alpha=0.15)

plt.vlines(x = n, ymin = 0, ymax = 105, color = 'blue', label = 'Offered Traffic = n channels')
plt.ylim([10,100])
plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('Grade Of Service (% probability of a call being delayed for > {} sec)'.format(allowedDelay*3600))
plt.title("GOS vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.savefig("sim1.png")

plt.show()


In [ ]:
# ERROR
qError = np.clip(erlangCs,0,100)-qGOS_exp
trafficRange = offeredTraffic(simulationCallAmounts,meanCallDuration)

plt.figure(figsize=(8,5))
print(qError)
# plt.hist(qError, 35, density=True)
plt.vlines(x = n, ymin = 0, ymax = 30, color = 'red', label = 'Offered Traffic = n channels')
plt.plot(trafficRange, qError, label="Error")
plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('GOS Error (% probability of a call being delayed)')
plt.title("GOS Error vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.show()

qAbsError = np.abs(qError)
plt.figure(figsize=(8,5))
plt.vlines(x = n, ymin = 0, ymax = 30, color = 'red', label = 'Offered Traffic = n channels')
plt.ylim([0,28])
plt.plot(trafficRange, qAbsError)
plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('GOS Absolute Error (% probability of a call being delayed)')
plt.title("GOS Absolute Error vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.show()

In [ ]:
simulationCallAmounts = (np.linspace(46, 56, 21)/meanCallDuration).astype(np.int)
repetitionsPerSimulation = 500
print(simulationCallAmounts)

erlangCs = np.zeros(simulationCallAmounts.shape)
meanCallDelays_calc = np.zeros(simulationCallAmounts.shape)

meanCallDelays_exp = np.zeros(simulationCallAmounts.shape)
callDelayStdDevs_exp = np.zeros(simulationCallAmounts.shape)

allowedDelay = 0 # (in hours)
recheckDelay = 0 #20/3600 # (in hours)

print("# Calls\t\tTraffic:\tCalc. delay/call\tSim Mean delay/call\tdelay/call stdDev")
for i, numCalls in enumerate(simulationCallAmounts):

    A0 = offeredTraffic(numCalls, meanCallDuration)
    
    erlangCs[i] = erlangC_allowedDelayGOS(n, A0, meanCallDuration, allowedDelay=allowedDelay)
    
    meanCallDelays_calc[i] = meanCallDelay(n, A0, meanCallDuration, erlangCs[i])*3600

    _, _,  meanCallDelay_exp, callDelayStdDev_exp = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, recheckChannelsDelay=recheckDelay,  holdingDistrib='exp', repetitions=repetitionsPerSimulation)
    meanCallDelays_exp[i] = meanCallDelay_exp*3600
    callDelayStdDevs_exp[i] = callDelayStdDev_exp*3600
    print("{}\t\t{:.3f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}".format(numCalls, A0, meanCallDelays_calc[i], meanCallDelays_exp[i], callDelayStdDevs_exp[i]))

plt.figure(figsize=(12,6))
trafficRange = offeredTraffic(simulationCallAmounts,meanCallDuration)
plt.plot(trafficRange, np.clip(meanCallDelays_calc,0,100), label='Erlang C Calculated Delay', color="orange")
plt.errorbar(trafficRange, meanCallDelays_exp, yerr=callDelayStdDevs_exp, label="Monte Carlo with requeue (Exponential)", color="red")
plt.fill_between(trafficRange, meanCallDelays_exp+callDelayStdDevs_exp, meanCallDelays_exp-callDelayStdDevs_exp, interpolate=True, color="red", alpha=0.15)
# plt.xlim([30,60])
plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('Mean Delay/Call (seconds)')
plt.title("Mean Delay per Call vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.show()

duration1 = meanCallDelays_exp
error1 = callDelayStdDevs_exp

In [ ]:
erlangCs = np.zeros(simulationCallAmounts.shape)
meanCallDelays_calc = np.zeros(simulationCallAmounts.shape)

meanCallDelays_exp = np.zeros(simulationCallAmounts.shape)
callDelayStdDevs_exp = np.zeros(simulationCallAmounts.shape)

allowedDelay = 0 # (in hours)
recheckDelay = 5/3600 # (in hours)

print("# Calls\t\tTraffic:\tCalc. delay/call\tSim Mean delay/call\tdelay/call stdDev")
for i, numCalls in enumerate(simulationCallAmounts):

    A0 = offeredTraffic(numCalls, meanCallDuration)
    
    erlangCs[i] = erlangC_allowedDelayGOS(n, A0, meanCallDuration, allowedDelay=allowedDelay)
    
    meanCallDelays_calc[i] = meanCallDelay(n, A0, meanCallDuration, erlangCs[i])*3600

    _, _,  meanCallDelay_exp, callDelayStdDev_exp = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, recheckChannelsDelay=recheckDelay,  holdingDistrib='exp', repetitions=repetitionsPerSimulation)
    meanCallDelays_exp[i] = meanCallDelay_exp*3600
    callDelayStdDevs_exp[i] = callDelayStdDev_exp*3600
    print("{}\t\t{:.3f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}".format(numCalls, A0, meanCallDelays_calc[i], meanCallDelays_exp[i], callDelayStdDevs_exp[i]))

plt.figure(figsize=(12,6))
trafficRange = offeredTraffic(simulationCallAmounts,meanCallDuration)
plt.plot(trafficRange, np.clip(meanCallDelays_calc,0,100), label='Erlang C Calculated Delay', color="orange")
plt.errorbar(trafficRange, meanCallDelays_exp, yerr=callDelayStdDevs_exp, label="Monte Carlo with requeue (Exponential)", color="red")
plt.fill_between(trafficRange, meanCallDelays_exp+callDelayStdDevs_exp, meanCallDelays_exp-callDelayStdDevs_exp, interpolate=True, color="red", alpha=0.15)
plt.ylim(bottom=0)
plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('Mean Delay/Call (seconds)')
plt.title("Mean Delay per Call vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.show()

duration2 = meanCallDelays_exp
error2 = callDelayStdDevs_exp

In [ ]:
plt.plot(trafficRange, duration1, label="no delay", color='red')
plt.fill_between(trafficRange, duration1+error1, duration1-error1, interpolate=True, color="red", alpha=0.18)

plt.plot(trafficRange, duration2, label="5s delay")
plt.fill_between(trafficRange, duration2+error2, duration2-error2, interpolate=True, color="blue", alpha=0.18)

# plt.ylim([0,5])
plt.legend()
plt.show()
print(duration1.mean(), duration2.mean())
print(error1.mean(), error2.mean())

In [ ]:
with open('block.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(["# Calls", "Offered Traffic", "Erlang B:", "Sim GOS (exponential):", "Sim std dev (exponential):", "Repetitions per simulation = "+str(repetitionsPerSimulation)])
    # (numCalls, A0, erlangBs[i], GOS_exp[i], stdDevs_exp[i],
    for i, numCalls in enumerate(simulationCallAmounts):
        writer.writerow([numCalls, A0s[i], erlangBs[i], GOS_exp[i], stdDevs_exp[i]])